In [1]:
from utils import load_model_and_files, get_model_details, reconstruct_word, merge_results, preprocess_text, preprocess_stopwords, preprocess_lemmatization, traduci_output, process_text_with_models, process_emotions_and_translate

In [2]:
# 1. Recupera i dettagli dei modelli
model_details = get_model_details()

# 2. Assegna manualmente le tuple a  separate
(model_1_id, model_1_type, model_1_files), (model_2_id, model_2_type, model_2_files), (model_3_id, model_3_type, model_3_files), (model_4_id, model_4_type, model_4_files) = model_details

# 3. Usa i dettagli per caricare i modelli
print(f"Caricamento modello 1: {model_1_id}")
model_1_pipeline = load_model_and_files(model_1_id, model_type=model_1_type)

print(f"Caricamento modello 2: {model_2_id}")
model_2_pipeline = load_model_and_files(model_2_id, model_type=model_2_type)

print(f"Caricamento modello 3: {model_3_id}")
model_3_pipeline = load_model_and_files(model_3_id, model_type=model_3_type)

print(f"Caricamento modello 4: {model_4_id}")
model_4_pipeline = load_model_and_files(model_4_id, model_type=model_4_type)


Caricamento modello 1: osiria/bert-italian-uncased-ner


Device set to use cpu


Caricamento modello 2: IVN-RIN/MedPsyNIT


Device set to use cpu


Caricamento modello 3: SamLowe/roberta-base-go_emotions


Device set to use cpu


Caricamento modello 4: Helsinki-NLP/opus-mt-it-en


c:\Users\lore9\Documents\GitHub\data_semantics\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [3]:
text = "Ciao Giulia, devo prendere un volo transoceanico per la prima volta nella mia vita tra due settimane e vivo nel terrore che possa cadere. Boeing non fa aerei molto sicuri a mio avviso. Devo andare da londra a parigi per lavoro. Per l'ansia prenderò un lexotan"

In [5]:
from pymongo import MongoClient

# Connetti al database MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["data_semantics_prova_6"]
collection = db["post"]
autori_collection = db["autori"]  # Collezione che contiene gli autori

DIMENSIONE_BLOCCO = 50

def recupera_autori():
    # Recupera la lista di autori dal database
    autori_cursor = autori_collection.find()
    autori = [autore["author"] for autore in autori_cursor]  # Supponendo che il campo del nome dell'autore sia "nome"
    return autori

def analisi_semantica(text, autori):
    # Supponendo che i risultati siano già ottenuti
    risultato_ner = process_text_with_models(text, model_1_pipeline, model_2_pipeline, model_1_id, model_2_id)
    risultato_sentiment = process_emotions_and_translate(text, model_4_pipeline, model_3_pipeline)[0]

    # Filtrare i sentimenti per raggiungere uno score cumulativo di almeno 0.95
    selected_sentiments = []
    cumulative_score = 0

    for sentiment in risultato_sentiment:
        if cumulative_score >= 0.95:
            break
        selected_sentiments.append(sentiment)
        cumulative_score += sentiment['score']

    # Controlla se la lista non è vuota
    if risultato_ner:
        # Itera sulla lista e converte i punteggi 'score' in float64
        for item in risultato_ner:
            item["score"] = float(item["score"])  # Converte np.float32 in float64
            # Se l'entità è un autore, etichettala come "PER"
            if item["word"] in autori:
                item["label"] = "PER"
    #else:
    #    print("La lista 'ner' è vuota!")

    risultato_sentiment = {item['label']: item['score'] for item in risultato_sentiment}

    selected_sentiments = {item['label']: item['score'] for item in selected_sentiments}

    # Creazione del dizionario
    data_semantic_analysis = {
        "ner": risultato_ner,
        "sentiment_analysis_relevant": selected_sentiments,
        "sentiment_analysis_full": risultato_sentiment
    }
    return data_semantic_analysis

# Recupera la lista di autori una volta all'inizio
autori = recupera_autori()

In [6]:
analisi_semantica(text,autori)

{'ner': [{'word': 'giulia', 'entity': 'PER', 'score': 0.5550692677497864},
  {'word': 'boeing', 'entity': 'ORG', 'score': 0.9962664246559143},
  {'word': 'londra', 'entity': 'LOC', 'score': 0.9963070154190063},
  {'word': 'parigi', 'entity': 'LOC', 'score': 0.9960122108459473},
  {'word': 'lexota', 'entity': 'MISC', 'score': 0.8247328996658325},
  {'word': 'lexotan',
   'entity': 'TRATTAMENTO FARMACOLOGICO (B)',
   'score': 0.96161949634552}],
 'sentiment_analysis_relevant': {'paura': 0.7363895177841187,
  'nervosismo': 0.203889399766922,
  'neutrale': 0.13923588395118713},
 'sentiment_analysis_full': {'paura': 0.7363895177841187,
  'nervosismo': 0.203889399766922,
  'neutrale': 0.13923588395118713,
  'approvazione': 0.03130808472633362,
  'tristezza': 0.028943508863449097,
  'realizzazione': 0.0280913058668375,
  'cura': 0.015214819461107254,
  'delusione': 0.012415895238518715,
  'gioia': 0.011776150204241276,
  'eccitazione': 0.010566255077719688,
  'confusione': 0.01009079627692699

In [17]:
risultato_ner

{'giulia': {0.5550692677497864, 'PER'},
 'boeing': {0.9962664246559143, 'ORG'},
 'londra': {0.9963070154190063, 'LOC'},
 'parigi': {0.9960122108459473, 'LOC'},
 'lexota': {0.8247328996658325, 'MISC'},
 'lexotan': {0.96161949634552, 'TRATTAMENTO FARMACOLOGICO (B)'}}